In [ ]:
!pip install openpyxl
!pip install huggingface_hub
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, MT5ForConditionalGeneration, MT5Tokenizer
import torch
import numpy as np
import json
import pandas as pd
from huggingface_hub import login
import os
import pandas as pd
import openai
import string
import ast
import re
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
login()
nltk.download('punkt')
nltk.download('punkt_tab')
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [ ]:
# @title model setting
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained('KrithikV/MedMobile')
model = AutoModelForCausalLM.from_pretrained(
        'KrithikV/MedMobile',
        torch_dtype=torch.bfloat16,
        device_map="auto")

translator_model_size = "large"
translator_en2fa_model_name = f"persiannlp/mt5-{translator_model_size}-parsinlu-translation_en_fa"
translator_en2fa_tokenizer = MT5Tokenizer.from_pretrained(translator_en2fa_model_name)
translator_en2fa_model = MT5ForConditionalGeneration.from_pretrained(translator_en2fa_model_name,torch_dtype=torch.bfloat16).to(device)

translator_fa2en_model_name = f"persiannlp/mt5-{translator_model_size}-parsinlu-opus-translation_fa_en"
translator_fa2en_tokenizer = MT5Tokenizer.from_pretrained(translator_fa2en_model_name)
translator_fa2en_model = MT5ForConditionalGeneration.from_pretrained(translator_fa2en_model_name,torch_dtype=torch.bfloat16).to(device)

model.eval()
translator_en2fa_model.eval()
translator_fa2en_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


pytorch_model.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=Fals

In [ ]:
# @title Utility
def translate_chunk_en2fa(input_string, **generator_args):
    input_ids = translator_en2fa_tokenizer.encode(input_string, return_tensors="pt").to(device)
    res = translator_en2fa_model.generate(input_ids, **generator_args)
    output = translator_en2fa_tokenizer.batch_decode(res, skip_special_tokens=True)
    return output

def translate_chunk_fa2en(input_string, **generator_args):
    input_ids = translator_fa2en_tokenizer.encode(input_string, return_tensors="pt").to(device)
    res = translator_fa2en_model.generate(input_ids, **generator_args)
    output = translator_fa2en_tokenizer.batch_decode(res, skip_special_tokens=True)
    return output

def translate_en2fa(text, chunk_length = 10):
    words = word_tokenize(text)
    chunks = []
    translation = ""
    for i in range(0, len(words), chunk_length):
        chunks.append(translate_chunk_en2fa(" ".join(words[i:i + chunk_length])))
    for chunk in chunks:
        translation = translation + " " + chunk[0]
    return translation

def translate_fa2en(text, chunk_length = 10):
    words = text.split()
    chunks = []
    translation = ""
    for i in range(0, len(words), chunk_length):
        chunks.append(translate_chunk_fa2en(" ".join(words[i:i + chunk_length])))
    for chunk in chunks:
        translation = translation + " " + chunk[0]
    return translation

def get_AI_answer(Prompt,Question):
    content = Prompt+Question
    messages = [{"role": "user", "content": f"{content}"}]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to('cuda:0')
    with torch.no_grad():
         outputs = model.generate(inputs, max_new_tokens=1024,temperature=0.0)
         text = tokenizer.batch_decode(outputs)[0]
         return translate_en2fa(text.split("<|assistant|>")[-1])

def append_record_to_excel(file_path,question,answer,source,AI_answer):
    new_record = {
        'Question': question,
        'answer': answer,
        'Sources': source,
        'AI_answer': AI_answer
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)


In [ ]:
Prompt = "شما یک پزشک حاذق هستید و در حوزه پزشکی دانش بسیار بالایی دارید لطفا هر آنچه در مورد پرسش مطرح شده میدانید بگویید"+'\n'+'پرسش:'
Question = """
علائم و نشانه‌های سندرم کوبویید چیست؟
"""
answer = get_AI_answer(Prompt,Question)
print(answer)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
df = pd.read_excel('test.xlsx')
Prompt = "شما یک پزشک حاذق هستید و در حوزه پزشکی دانش بسیار بالایی دارید لطفا هر آنچه در مورد پرسش مطرح شده میدانید بگویید"+'\n'+'پرسش:'
i = 0
for index,row in df.iterrows():
        if index <= 1976:
           i += 1
           continue
        question = row['Question']
        answer = row['Answer']
        source = row['Source']
        AI_answer = get_AI_answer(Prompt,question)
        file_path = 'medparsinlu.xlsx'
        append_record_to_excel(file_path,question,answer,source,AI_answer)
        i += 1
        print(i)
        print('============================================================')


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


1978
1979


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


1980


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


...

1989


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


1990


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 